In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib openai
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.8 MB/s eta 0:00:00
  Installing build dependencies

#### Open AI API Key Set

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

#### Dataset Creation

In [ ]:
import openai

def prompt_model(prompt_list, model="gpt-3.5-turbo"):
  return openai.ChatCompletion.create(model=model, messages=prompt_list)

def extract_content(openai_response_object):
  return openai_response_object["choices"][0]["message"]["content"]

In [ ]:
datagen_prompts = [
    {"role" : "system", "content" : "You are a pirate translator. You translate English utterances into Pirate."},
    {"role" : "user", "content" : "Please generate a list of 10 English to Pirate utterance pairs."},
]

In [ ]:
dataset_generation = prompt_model(datagen_prompts)
print(extract_content(dataset_generation))

1. English: Hello pirate, how are you?
   Pirate: Ahoy matey, how be ye?

2. English: I like your pirate hat.
   Pirate: Yarr, I fancy yer cap'n's hat.

3. English: Can you show me how to tie a knot?
   Pirate: Can ye teach me how to tie a knot, me hearty?

4. English: I am going to the market.
   Pirate: I be headin' to the market, ho!

5. English: May I borrow your telescope?
   Pirate: Aye, can I borrow yer spyglass, ye scurvy dog?

6. English: Please bring me a glass of water.
   Pirate: Arrr, fetch me a mug of fresh water, ye landlubber!

7. English: The ship is coming towards us.
   Pirate: Thar she blows, the ship be comin' our way!

8. English: I need to fix my ship's sail.
   Pirate: Me ship's sail needs mendin', arrr!

9. English: Can you help me find the treasure?
   Pirate: Avast, can ye lend me a hand in findin' the booty?

10. English: I am tired and need some sleep.
    Pirate: Arrr, I be wearied out and need some shuteye, matey.


In [ ]:
def retrieve_token_usage(open_ai_response):
  return sum([tokens for tokens in open_ai_response["usage"].values()])

f"We used {retrieve_token_usage(dataset_generation)} tokens"

'We used 644 tokens'

In [ ]:
def extract_english_and_pirate(openai_dataset_content):
  lines = openai_dataset_content.split("\n\n")

  converted_lines = []

  for line in lines:
    line_dict = {"english" : "", "pirate" : ""}  
    english, pirate = line.split("\n")

    english = english.split(":")[-1][1:]
    pirate = pirate.split(":")[-1][1:]

    line_dict["english"] = english
    line_dict["pirate"] = pirate

    converted_lines.append(line_dict)

  return converted_lines

In [ ]:
converted_lines = extract_english_and_pirate(extract_content(dataset_generation))

In [ ]:
converted_lines

[{'english': 'Hello pirate, how are you?', 'pirate': 'Ahoy matey, how be ye?'},
 {'english': 'I like your pirate hat.',
  'pirate': "Yarr, I fancy yer cap'n's hat."},
 {'english': 'Can you show me how to tie a knot?',
  'pirate': 'Can ye teach me how to tie a knot, me hearty?'},
 {'english': 'I am going to the market.',
  'pirate': "I be headin' to the market, ho!"},
 {'english': 'May I borrow your telescope?',
  'pirate': 'Aye, can I borrow yer spyglass, ye scurvy dog?'},
 {'english': 'Please bring me a glass of water.',
  'pirate': 'Arrr, fetch me a mug of fresh water, ye landlubber!'},
 {'english': 'The ship is coming towards us.',
  'pirate': "Thar she blows, the ship be comin' our way!"},
 {'english': "I need to fix my ship's sail.",
  'pirate': "Me ship's sail needs mendin', arrr!"},
 {'english': 'Can you help me find the treasure?',
  'pirate': "Avast, can ye lend me a hand in findin' the booty?"},
 {'english': 'I am tired and need some sleep.',
  'pirate': 'Arrr, I be wearied o

In [ ]:
import pandas as pd

dataset_dataframe = pd.DataFrame(converted_lines)
dataset_dataframe

,english,pirate
0,"Hello pirate, how are you?","Ahoy matey, how be ye?"
1,I like your pirate hat.,"Yarr, I fancy yer cap'n's hat."
2,Can you show me how to tie a knot?,"Can ye teach me how to tie a knot, me hearty?"
3,I am going to the market.,"I be headin' to the market, ho!"
4,May I borrow your telescope?,"Aye, can I borrow yer spyglass, ye scurvy dog?"
5,Please bring me a glass of water.,"Arrr, fetch me a mug of fresh water, ye landlu..."
6,The ship is coming towards us.,"Thar she blows, the ship be comin' our way!"
7,I need to fix my ship's sail.,"Me ship's sail needs mendin', arrr!"
8,Can you help me find the treasure?,"Avast, can ye lend me a hand in findin' the bo..."
9,I am tired and need some sleep.,"Arrr, I be wearied out and need some shuteye, ..."


In [ ]:
from datasets import Dataset, DatasetDict

dataset = Dataset.from_pandas(dataset_dataframe)

train_test_dataset = dataset.train_test_split(test_size=0.2)
# Split the 10% test + valid in half test, half valid
test_valid = train_test_dataset['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
train_test_valid_dataset = DatasetDict({
    'train': train_test_dataset['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

In [ ]:
train_test_valid_dataset

DatasetDict({
    train: Dataset({
        features: ['english', 'pirate'],
        num_rows: 8
    })
    test: Dataset({
        features: ['english', 'pirate'],
        num_rows: 1
    })
    valid: Dataset({
        features: ['english', 'pirate'],
        num_rows: 1
    })
})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
DATASET_NAME = ""

train_test_valid_dataset.push_to_hub(DATASET_NAME)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-3b", 
    torch_dtype=torch.float16,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/tokenizer")

for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

from peft import LoraConfig, get_peft_model 

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

trainable params: 2457600 || all params: 3005015040 || trainable%: 0.08178328451893539


In [ ]:
def create_prompt(pirate, english):
  prompt_template = f"### ENGLISH\n{english}\n\n### PIRATE\n{pirate}</s>"
  return prompt_template

mapped_dataset = train_test_valid_dataset.map(lambda samples: tokenizer(create_prompt(samples['pirate'], samples['english'])))

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
import transformers

trainer = transformers.Trainer(
    model=model, 
    train_dataset=mapped_dataset["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4, 
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=50,
        learning_rate=1e-3, 
        fp16=True,
        logging_steps=1, 
        output_dir='outputs',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.023500
2,2.040300
3,2.037200
4,2.031200
5,2.020600
6,1.999300
7,1.979500
8,1.957600
9,1.923000
10,1.890500


TrainOutput(global_step=50, training_loss=0.8928632732480765, metrics={'train_runtime': 21.3173, 'train_samples_per_second': 37.528, 'train_steps_per_second': 2.346, 'total_flos': 198358615572480.0, 'train_loss': 0.8928632732480765, 'epoch': 50.0})

In [ ]:
model = model.merge_and_unload()

In [ ]:
HUGGING_FACE_USER_NAME = ""
model_name = ""

model.push_to_hub(f"{HUGGING_FACE_USER_NAME}/{model_name}", use_auth_token=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/6.01G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/c-s-ale/english_to_pirate_model/commit/e2d92babc5339ae1151a1d5fbfccf15166d2535a', commit_message='Upload BloomForCausalLM', commit_description='', oid='e2d92babc5339ae1151a1d5fbfccf15166d2535a', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub(f"{HUGGING_FACE_USER_NAME}/{model_name}")

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/c-s-ale/english_to_pirate_model/commit/368dafff5c22d890a86c959a18a3d6e0758a47f0', commit_message='Upload tokenizer', commit_description='', oid='368dafff5c22d890a86c959a18a3d6e0758a47f0', pr_url=None, pr_revision=None, pr_num=None)